# Neo4j
Neo4j is one of the popular Graph Databases and Cypher Query Language (CQL).

A graph is a pictorial representation of a set of objects where some pairs of objects are connected by links. It is composed of two elements - nodes (vertices) and relationships (edges).

Graph database is a database used to model the data in the form of graph. In here, the nodes of a graph depict the entities while the relationships depict the association of these nodes.

It is Flexible data model , High availability, Cypher query language, No joins

In [1]:
from neo4j import GraphDatabase
import pandas as pd

In [2]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "1234"

driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

## Creating a nodes.

You can create a node in Neo4j using the CREATE clause. 

Create a single node\
Create multiple nodes\
Create a node with a label\
Create a node with multiple labels\
Create a node with properties\
Returning the created node

In [3]:
q1 = 'create constraint on (p:Person) assert p.name is unique;'
session.run(q1)
q1 = 'CREATE (ee:Person { name: "Emil", from: "Sweden"})'
session.run(q1)

# Print data

In [4]:
q1 = 'MATCH (ee:Person) RETURN ee;'

values = [record.data() for record in session.run(q1)]
print(values)

[{'ee': {'name': 'Emil', 'from': 'Sweden'}}]


In [5]:
for value in values:
    value = value.values()
    print(pd.DataFrame(value))

   name    from
0  Emil  Sweden


## Creating Relationships
We can create a relationship using the CREATE clause. We will specify relationship within the square braces “[ ]” depending on the direction of the relationship it is placed between hyphen “ - ” and arrow “ → ” as shown in the following syntax.

In [6]:
q1 = 'MATCH (ee:Person) WHERE ee.name = "Emil" \
    CREATE (js:Person { name: "Johan", from: "Sweden", learn: "surfing" }) \
    create (ee) -[r:KNOWS{since:1999}] -> (js)'
session.run(q1)

q1 = 'MATCH (ee:Person{name : "Emil"})- [r]-(js:Person{name:"Johan"}) RETURN ee.name, type(r),js.name;'

values = [record.data() for record in session.run(q1)]
print(values)

[{'ee.name': 'Emil', 'type(r)': 'KNOWS', 'js.name': 'Johan'}]


## CREATE more

In [7]:
q1 = 'MATCH (ee:Person) WHERE ee.name = "Emil" \
MATCH (js:Person) where js.name = "Johan" \
CREATE (ir:Person { name: "Ian", from: "England", title: "author" }), \
(rvb:Person { name: "Rik", from: "Belgium", pet: "Orval" }), \
(ally:Person { name: "Allison", from: "California", hobby: "surfing" }), \
(ee)-[:KNOWS]->(ir), \
(js)-[:KNOWS]->(ir),(js)-[:KNOWS]->(rvb), \
(ir)-[:KNOWS]->(js),(ir)-[:KNOWS]->(ally), \
(rvb)-[:KNOWS]->(ally)'

session.run(q1)

In [8]:
q1 ="  match (n) return n"
values = [record.data() for record in session.run(q1)]

for value in values:
    print(value)

{'n': {'name': 'Johan', 'from': 'Sweden', 'learn': 'surfing'}}
{'n': {'name': 'Ian', 'from': 'England', 'title': 'author'}}
{'n': {'name': 'Rik', 'from': 'Belgium', 'pet': 'Orval'}}
{'n': {'name': 'Allison', 'from': 'California', 'hobby': 'surfing'}}
{'n': {'name': 'Emil', 'from': 'Sweden'}}


In [9]:
d = []
for value in values:
    value = value.values()
    for valu in value:
        d.append(valu)
df = pd.DataFrame(d)
df

,name,from,learn,title,pet,hobby
0,Johan,Sweden,surfing,NaN,NaN,NaN
1,Ian,England,NaN,author,NaN,NaN
2,Rik,Belgium,NaN,NaN,Orval,NaN
3,Allison,California,NaN,NaN,NaN,surfing
4,Emil,Sweden,NaN,NaN,NaN,NaN


In [10]:
q1 = 'MATCH (ee:Person)- [r]-(js:Person) RETURN ee.name, type(r),js.name;'

values = [record.data() for record in session.run(q1)]

for value in values:
    print(value)

{'ee.name': 'Johan', 'type(r)': 'KNOWS', 'js.name': 'Ian'}
{'ee.name': 'Johan', 'type(r)': 'KNOWS', 'js.name': 'Rik'}
{'ee.name': 'Johan', 'type(r)': 'KNOWS', 'js.name': 'Ian'}
{'ee.name': 'Johan', 'type(r)': 'KNOWS', 'js.name': 'Emil'}
{'ee.name': 'Ian', 'type(r)': 'KNOWS', 'js.name': 'Allison'}
{'ee.name': 'Ian', 'type(r)': 'KNOWS', 'js.name': 'Johan'}
{'ee.name': 'Ian', 'type(r)': 'KNOWS', 'js.name': 'Johan'}
{'ee.name': 'Ian', 'type(r)': 'KNOWS', 'js.name': 'Emil'}
{'ee.name': 'Rik', 'type(r)': 'KNOWS', 'js.name': 'Allison'}
{'ee.name': 'Rik', 'type(r)': 'KNOWS', 'js.name': 'Johan'}
{'ee.name': 'Allison', 'type(r)': 'KNOWS', 'js.name': 'Ian'}
{'ee.name': 'Allison', 'type(r)': 'KNOWS', 'js.name': 'Rik'}
{'ee.name': 'Emil', 'type(r)': 'KNOWS', 'js.name': 'Ian'}
{'ee.name': 'Emil', 'type(r)': 'KNOWS', 'js.name': 'Johan'}


## Pattern matching
Describe what to find in the graph

For instance, a pattern can be used to find Emil's friends:

In [11]:
q1 = 'MATCH (ee:Person)-[:KNOWS]-(friends) \
WHERE ee.name = "Emil" RETURN ee, friends'

values = [record.data() for record in session.run(q1)]

for value in values:
    print(value)

{'ee': {'name': 'Emil', 'from': 'Sweden'}, 'friends': {'name': 'Ian', 'from': 'England', 'title': 'author'}}
{'ee': {'name': 'Emil', 'from': 'Sweden'}, 'friends': {'name': 'Johan', 'from': 'Sweden', 'learn': 'surfing'}}


In [12]:
q1 = 'MATCH (js:Person)-[:KNOWS]-()-[:KNOWS]-(surfer) \
WHERE js.name = "Johan" AND surfer.hobby = "surfing" \
RETURN DISTINCT surfer'

values = [record.data() for record in session.run(q1)]

for value in values:
    print(value)

{'surfer': {'name': 'Allison', 'from': 'California', 'hobby': 'surfing'}}


## Set Clause
Using Set clause, you can add new properties to an existing Node or Relationship, and also add or update existing Properties values.

In [13]:
q1 = 'MATCH (n1:Person{name: "Emil"}) \
SET n1.age = 18 \
RETURN n1'

session.run(q1)

q1 = 'MATCH (n1:Person) WHERE n1.name = "Emil" RETURN n1;'

values = [record.data() for record in session.run(q1)]
print(values)

[{'n1': {'name': 'Emil', 'from': 'Sweden', 'age': 18}}]


## Setting Multiple Properties
In the same way, you can create multiple properties in a node using the Set clause. To do so, you need to specify these key value pairs with commas.

In [14]:
q1 = 'MATCH (n1:Person{name: "Emil"}) \
SET n1.from = "USA", n1.POB= "UK" \
RETURN n1'

session.run(q1)

q1 = 'MATCH (n1:Person) WHERE n1.name = "Emil" RETURN n1;'

values = [record.data() for record in session.run(q1)]
print(values)

[{'n1': {'name': 'Emil', 'POB': 'UK', 'from': 'USA', 'age': 18}}]


## Removing a Property
You can remove an existing property by passing NULL as value to it.

In [15]:
q1 = 'MATCH (n1:Person{name: "Emil"}) \
SET n1.age = NULL \
RETURN n1'

session.run(q1)

q1 = 'MATCH (n1:Person) WHERE n1.name = "Emil" RETURN n1;'

values = [record.data() for record in session.run(q1)]
print(values)

[{'n1': {'name': 'Emil', 'POB': 'UK', 'from': 'USA'}}]


## Deleting All Nodes and Relationships
Following is the query to delete all the nodes and the relationships in the database using the DELETE clause.

In [16]:
q1 = 'MATCH (n1) DETACH DELETE n1'

# session.run(q1)

## Deleting a Particular Node
To delete a particular node, you need to specify the details of the node in the place of “n” in the above query.

In [17]:
q1 = 'MATCH (ee:Person) WHERE ee.name = "Emil" DETACH DELETE ee;'
session.run(q1)

q1 = 'MATCH (ee:Person) WHERE ee.name = "Emil" RETURN ee;'

values = [record.data() for record in session.run(q1)]
print(values)

[]


## Remove Clause
The REMOVE clause is used to remove properties and labels from graph elements (Nodes or Relationships).

In [18]:
q1 = 'MATCH (n1:Person) WHERE n1.name = "Johan" RETURN n1;'

values = [record.data() for record in session.run(q1)]
print(values)

q1 = 'MATCH (n1:Person{name: "Johan" }) \
REMOVE n1.learn \
RETURN n1'

session.run(q1)

q1 = 'MATCH (n1:Person) WHERE n1.name = "Johan" RETURN n1;'

values = [record.data() for record in session.run(q1)]
print(values)

[{'n1': {'name': 'Johan', 'from': 'Sweden', 'learn': 'surfing'}}]
[{'n1': {'name': 'Johan', 'from': 'Sweden'}}]


In [19]:
q1 = 'MATCH (n1:Person) WHERE n1.name = "Johan" RETURN n1;'

values = [record for record in session.run(q1)]
print(values)

q1 = 'MATCH (n1:Person{name: "Johan" }) \
REMOVE n1:Person \
RETURN n1'

session.run(q1)

q1 = 'MATCH (n1) WHERE n1.name = "Johan" RETURN n1;'

values = [record for record in session.run(q1)]
print(values)

[<Record n1=<Node id=1 labels=frozenset({'Person'}) properties={'name': 'Johan', 'from': 'Sweden'}>>]
[<Record n1=<Node id=1 labels=frozenset() properties={'name': 'Johan', 'from': 'Sweden'}>>]


# Count Function
The count() function is used to count the number of rows.

In [20]:
q1 = 'Match (n:Person) \
RETURN count(n)'

values = [record.data() for record in session.run(q1)]
print(values)

[{'count(n)': 3}]


In [21]:
q1 = 'Match (n:Person{name : "Ian"}) -[:KNOWS] ->(x) \
RETURN count( distinct x)'

values = [record.data() for record in session.run(q1)]
print(values)

[{'count( distinct x)': 2}]


In [22]:
q1 = 'Match (n:Person{name : "Ian"}) -[r] ->(x) \
RETURN type(r), count(x)'

values = [record.data() for record in session.run(q1)]
print(values)

[{'type(r)': 'KNOWS', 'count(x)': 2}]


In [23]:
q1 = 'Match (n:Person{name : "Ian"}) -[r] ->(x) \
RETURN type(r), labels(x), count(x)'

values = [record.data() for record in session.run(q1)]
print(values)

[{'type(r)': 'KNOWS', 'labels(x)': ['Person'], 'count(x)': 1}, {'type(r)': 'KNOWS', 'labels(x)': [], 'count(x)': 1}]


### Count used to group

In [24]:
q1 = 'Match (n:Person) \
RETURN n.labels, count(n)'

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'n.labels': None, 'count(n)': 3}


# Order By Clause
arrange the result data in order using the ORDER BY clause

In [25]:
q1 = 'Match (n:Person) \
RETURN n order by n.name'

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'n': {'name': 'Allison', 'from': 'California', 'hobby': 'surfing'}}
{'n': {'name': 'Ian', 'from': 'England', 'title': 'author'}}
{'n': {'name': 'Rik', 'from': 'Belgium', 'pet': 'Orval'}}


In [26]:
q1 = 'create constraint on (p:player) assert p.name is unique;'
session.run(q1)

q1 = 'CREATE(Dhawan:player{name:"shikar Dhawan", YOB: 1985, runs:363, country: "India"}) \
CREATE(Jonathan:player{name:"Jonathan Trott", YOB:1981, runs:229, country:"South Africa"}) \
CREATE(Sangakkara:player{name:"Kumar Sangakkara", YOB:1977, runs:222, country:"Srilanka"}) \
CREATE(Rohit:player{name:"Rohit Sharma", YOB: 1987, runs:177, country:"India"}) \
CREATE(Virat:player{name:"Virat Kohli", YOB: 1988, runs:176, country:"India"})'
session.run(q1)

q1 = 'Match (n:player) \
RETURN n order by n.runs'

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'n': {'name': 'Virat Kohli', 'country': 'India', 'YOB': 1988, 'runs': 176}}
{'n': {'name': 'Rohit Sharma', 'country': 'India', 'YOB': 1987, 'runs': 177}}
{'n': {'name': 'Kumar Sangakkara', 'country': 'Srilanka', 'YOB': 1977, 'runs': 222}}
{'n': {'name': 'Jonathan Trott', 'country': 'South Africa', 'YOB': 1981, 'runs': 229}}
{'n': {'name': 'shikar Dhawan', 'country': 'India', 'YOB': 1985, 'runs': 363}}


In [27]:
q1 = 'Match (n:player) \
RETURN n order by n.runs desc'

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'n': {'name': 'shikar Dhawan', 'country': 'India', 'YOB': 1985, 'runs': 363}}
{'n': {'name': 'Jonathan Trott', 'country': 'South Africa', 'YOB': 1981, 'runs': 229}}
{'n': {'name': 'Kumar Sangakkara', 'country': 'Srilanka', 'YOB': 1977, 'runs': 222}}
{'n': {'name': 'Rohit Sharma', 'country': 'India', 'YOB': 1987, 'runs': 177}}
{'n': {'name': 'Virat Kohli', 'country': 'India', 'YOB': 1988, 'runs': 176}}


In [28]:
q1 = 'Match (n:player) \
RETURN n order by n.YOB, n.runs'

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'n': {'name': 'Kumar Sangakkara', 'country': 'Srilanka', 'YOB': 1977, 'runs': 222}}
{'n': {'name': 'Jonathan Trott', 'country': 'South Africa', 'YOB': 1981, 'runs': 229}}
{'n': {'name': 'shikar Dhawan', 'country': 'India', 'YOB': 1985, 'runs': 363}}
{'n': {'name': 'Rohit Sharma', 'country': 'India', 'YOB': 1987, 'runs': 177}}
{'n': {'name': 'Virat Kohli', 'country': 'India', 'YOB': 1988, 'runs': 176}}


# Limit Clause
The limit clause is used to limit the number of rows in the output.

In [29]:
q1 = 'Match (n:player) \
RETURN n order by n.YOB, n.runs limit 3'

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'n': {'name': 'Kumar Sangakkara', 'country': 'Srilanka', 'YOB': 1977, 'runs': 222}}
{'n': {'name': 'Jonathan Trott', 'country': 'South Africa', 'YOB': 1981, 'runs': 229}}
{'n': {'name': 'shikar Dhawan', 'country': 'India', 'YOB': 1985, 'runs': 363}}


In [30]:
q1 = 'Match (n:player) \
     with n \
     order by n.runs desc limit 3 \
     RETURN collect(n.name) '

values = [record.data() for record in session.run(q1)]
for  value in values:
    print(value)

{'collect(n.name)': ['shikar Dhawan', 'Jonathan Trott', 'Kumar Sangakkara']}
